# PageRank на Spark RDD

### Шаг №1
Создайте SparkContext

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "natalya.pritykovskaya Spark RDD Pagerank") 

sc = SparkContext(conf=conf)

### Шаг №2
1. Прочитайте граф из файла `/lectures/lecture01/data/graph.tsv`
2. Создайте RDD, в которой граф будет представлен парами вершин


In [3]:
raw_graph = sc.textFile("/lectures/lecture01/data/graph.tsv")
graph = raw_graph.map(lambda x: tuple(x.split("\t")))\
                 .distinct()\
                 .cache()
graph.collect()

[('A', 'B'),
 ('B', 'D'),
 ('A', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'D'),
 ('D', 'C')]

### Шаг №3
Создайте RDD с первоначальными pagerank всех уникальных вершин

In [4]:
num_vertices = graph.map(lambda x: x[0])\
                    .distinct()\
                    .count() 

ranks = graph.map(lambda x: x[0])\
             .distinct()\
             .map(lambda x: (x, 1 / num_vertices))
ranks.collect()

[('C', 0.25), ('A', 0.25), ('B', 0.25), ('D', 0.25)]

### Шаг №4
Создайте RDD, которая берет RDD с вершинами, объединяет ее с RDD с pagerank. В результате должна получится PairRDD, где ключ - это уникальная вершина, а значение - это все вершины, на которые она ссылаются и ее текущий pagerank

In [5]:
links = graph.groupByKey().mapValues(list).cache()

contributions = links.join(ranks)
contributions.collect()

[('A', (['B', 'C'], 0.25)),
 ('B', (['D'], 0.25)),
 ('C', (['A', 'B', 'D'], 0.25)),
 ('D', (['C'], 0.25))]

### Шаг №5
Реализуйте функцию, которая рассчитывает pagerank для всех вершин, на которые ссылается данная вершина. Функция должна быть итератором, который возвращает вершину и ее pagerank

In [6]:
def computeContribs(neighbours, pagerank):
    num = len(neighbours)
    
    for n in neighbours:
        yield (n, pagerank / num)

### Шаг №6
Обновите RDD с pagerank значениями, посчитанными с помощью функции из предыдущего шага

In [7]:
ranks = contributions.flatMap(lambda x: computeContribs(x[1][0], x[1][1]))\
                     .reduceByKey(lambda x, y: x + y)

ranks.collect()

[('B', 0.20833333333333331),
 ('A', 0.08333333333333333),
 ('C', 0.375),
 ('D', 0.3333333333333333)]

### Шаг №7
Напишите цикл, который проводит несколько итераций вычисления pagerank и на каждой печатает номер итерации и текущие pagerank

In [8]:
iterations = 10
raw_graph = sc.textFile("/lectures/lecture01/data/graph.tsv")
graph = raw_graph.map(lambda x: tuple(x.split("\t")))\
                 .distinct()\
                 .cache()
num_vertices = graph.map(lambda x: x[0])\
                    .distinct()\
                    .count() 
ranks = graph.map(lambda x: x[0])\
             .distinct()\
             .map(lambda x: (x, 1 / num_vertices))

for i in range(iterations):
    links = graph.groupByKey().mapValues(list).cache()
    contributions = links.join(ranks)
    ranks = contributions.flatMap(lambda x: computeContribs(x[1][0], x[1][1]))\
                         .reduceByKey(lambda x, y: x + y)
    print("Iteration {0}: current pagerank {1}".format(i, sorted(ranks.collect(), key=lambda x: x[0])))

Iteration 0: current pagerank [('A', 0.08333333333333333), ('B', 0.20833333333333331), ('C', 0.375), ('D', 0.3333333333333333)]
Iteration 1: current pagerank [('A', 0.125), ('B', 0.16666666666666666), ('C', 0.375), ('D', 0.3333333333333333)]
Iteration 2: current pagerank [('A', 0.125), ('B', 0.1875), ('C', 0.3958333333333333), ('D', 0.29166666666666663)]
Iteration 3: current pagerank [('A', 0.13194444444444445), ('B', 0.19444444444444445), ('C', 0.35416666666666663), ('D', 0.3194444444444444)]
Iteration 4: current pagerank [('A', 0.11805555555555554), ('B', 0.18402777777777776), ('C', 0.38541666666666663), ('D', 0.3125)]
Iteration 5: current pagerank [('A', 0.1284722222222222), ('B', 0.18749999999999997), ('C', 0.3715277777777778), ('D', 0.3125)]
Iteration 6: current pagerank [('A', 0.1238425925925926), ('B', 0.18807870370370372), ('C', 0.3767361111111111), ('D', 0.31134259259259256)]
Iteration 7: current pagerank [('A', 0.1255787037037037), ('B', 0.1875), ('C', 0.37326388888888884), (

### Шаг №8
Не забудьте остановить SparkContext

In [ ]:
sc.stop()